# Load and preprocess data 

In [1]:
stop_words = set(["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"])

In [2]:
from pathlib import Path
import re

def rm_stop(words):  # remove stop words from list of words
    return [w for w in words.split() if w not in stop_words]

def preprocess(w):  # remove non-ASCII and lower characters from w
    return rm_stop(re.sub("[^a-zA-Z ]", '', w).lower())

class InstaCities:
    def __iter__(self):
        for f in Path("../InstaCities1M/captions_resized_1M/cities_instagram").glob("**/*.txt"):
            yield preprocess(f.read_text())

# Word2Vec model

In [3]:
import gensim.models

sentences = InstaCities()
model = gensim.models.Word2Vec(sentences=sentences, vector_size=30, window=4, min_count=5, workers=8)
model.save("model_insta_cities")

In [5]:
from gensim.models import Word2Vec

model = Word2Vec.load("model_insta_cities")

UnpicklingError: could not find MARK

In [4]:
wv = model.wv
wv.most_similar('city', topn=10)

[('buildings', 0.822852611541748),
 ('building', 0.8174885511398315),
 ('town', 0.7865408658981323),
 ('downtown', 0.7828469276428223),
 ('bridge', 0.7779280543327332),
 ('skyline', 0.7747320532798767),
 ('view', 0.773141622543335),
 ('streets', 0.771625280380249),
 ('views', 0.7570635080337524),
 ('people', 0.7509516477584839)]

# Comparison with Google News model

In [5]:
import gensim.downloader as api
wv_news = api.load('word2vec-google-news-300')

In [6]:
from pprint import pprint

pprint(wv_news.most_similar(positive=["car", "sports", "hairstyle", "food", "toronto"], topn=5))
pprint(model.wv.most_similar(positive=["car", "sports", "hairstyle", "food", "toronto"], topn=5))

[('DeVillers_reports', 0.5512474179267883),
 ('limited_edition_MGTF', 0.5153472423553467),
 ('e_mail_mikenadel@sbcglobal.net', 0.5088345408439636),
 ('TAKE_TWO_An', 0.4983653426170349),
 ('KTEN_Deeda_Payton', 0.48801666498184204)]
[('cars', 0.7308588027954102),
 ('gym', 0.7111132740974426),
 ('business', 0.7088854312896729),
 ('fitness', 0.6985484957695007),
 ('cool', 0.6978669762611389)]


In [7]:
import numpy as np

top = 7

def similar(wv, *words):
    v = np.sum([wv.get_vector(w) for w in words], axis=0)
    return [w[0] for w in wv.similar_by_vector(v, topn=top)]

for w1, w2 in [("toronto", "wild"), ("happy", "family"), ("food", "healthy"), ("food", "sweet")]:
    print(f"Most similar words to {w1} + {w2}:")
    print(f"InstaCities: {similar(wv, w1, w2)}")
    print(f"Google News: {similar(wv_news, w1, w2)}\n")

print(f"Most similar words to food - healthy:")
print(f"InstaCities: {[w[0] for w in wv.similar_by_vector(wv.get_vector('food') - wv.get_vector('healthy'), topn=top)]}")
print(f"Google News: {[w[0] for w in wv_news.similar_by_vector(wv_news.get_vector('food') - wv_news.get_vector('healthy'), topn=top)]}")
print([w[0] for w in model.wv.most_similar(positive=['food'] , negative=['healthy'], topn=top)])

Most similar words to toronto + wild:
InstaCities: ['toronto', 'ontario', 'canada', 'spring', 'london', 'red', 'canadian']
Google News: ['toronto', 'wild', 'montreal', 'vancouver', 'blackpool', 'alberta', 'okc']

Most similar words to happy + family:
InstaCities: ['family', 'happy', 'friends', 'friendship', 'fun', 'happiness', 'love']
Google News: ['happy', 'family', 'glad', 'thankful', 'Said_Hirschbeck', 'friends', 'safe_Arlene_Deche']

Most similar words to food + healthy:
InstaCities: ['healthy', 'food', 'foods', 'diet', 'eat', 'health', 'tasty']
Google News: ['healthy', 'food', 'nutritious', 'healthier_tastier', 'nutritionally_sound', 'foods', 'nutritiously']

Most similar words to food + sweet:
InstaCities: ['food', 'delicious', 'yummy', 'tasty', 'yum', 'sweet', 'eat']
Google News: ['sweet', 'food', 'delicious', 'yummy', 'crunchy_salty', 'savory_delights', 'tasty']

Most similar words to food - healthy:
InstaCities: ['fooddrink', 'truman', 'chrisburkard', 'myswitzerland', 'potter'

In [74]:
print(wv.doesnt_match(["man", "woman", "kid", "dog"]))
print(wv_news.doesnt_match(["man", "woman", "kid", "dog"]))

dog
dog


In [13]:
print(wv.similarity('woman', 'man'))
print(wv_news.similarity('woman', 'man'))

0.78754294
0.76640123


# Caption representation

In [20]:
from collections import defaultdict

def repr(caption):
    words = preprocess(caption)
    words = [wv.get_vector(w) for w in words if w in wv.key_to_index]
    if len(words) > 0:
        v = np.sum(words, axis=0)
        v -= min(v)
        if max(v) != 0:
            v /= max(v)
        return v

n1, n2 = 0, 0

for dir_city in Path("../InstaCities1M/captions_resized_1M/cities_instagram").iterdir():
    for file_img in dir_city.iterdir():
        v = repr(file_img.read_text())
        if v is not None:
            f = train_file
            if n1 > 80_000:
                f = val_file
            elif n2 > 85_000:
                f = test_file
            np.savetxt(f"caption/{file_img.name}", v)

KeyboardInterrupt: 